In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import dotenv

from mtclient import MTClient
import mtglobals

In [3]:
client = MTClient()

Loading environment variables from .env
Using LIVE MTurk requester API
Your account balance is 4775.12


### (a) Load the stage-1 task XML data

In [4]:
# The question we ask the workers in the *first* stage is contained in this file.
dotenv.load_dotenv(mtglobals.dotenv_fpath, override=True)
stage1_html_fpath = os.getenv("STAGE1_HTML_FPATH")
# Wrap the html in the XML format required by MTurk
stage1_xml_fpath = mtglobals.gen_xml(stage1_html_fpath)
with open(stage1_xml_fpath, "r", encoding='utf-8') as f:
    stage1_question = f.read()

### (b) Set up the two requirements for the stage-1 task

(The first is that they're in the US, the second that they have *not* been assigned the `Workplace_Survey_Participant` qual before)

In [5]:
# Get the participation qual id from the .env file
#print(dotenv.dotenv_values(mtglobals.dotenv_fpath))
dotenv.load_dotenv(mtglobals.dotenv_fpath, override=True)
participation_qual_id = os.getenv("PARTICIPATION_QUAL_ID")
participation_qual_id

'3ACE7X5EFA17IX9O66OVG7YL1JYEN7'

In [6]:
# Example of using qualification to restrict responses to Workers who have had
# at least 80% of their assignments approved. See:
# http://docs.aws.amazon.com/AWSMechTurk/latest/AWSMturkAPI/ApiReference_QualificationRequirementDataStructureArticle.html#ApiReference_QualificationType-IDs
stage1_requirements = [
    {
        # MTurk-provided qual id for restricting by country
        'QualificationTypeId':"00000000000000000071",
        'Comparator':"EqualTo",
        'LocaleValues':[{
            'Country':"US"
        }]
    },
    {
        # Workplace_Survey_Participant qual id
        'QualificationTypeId': participation_qual_id,
        'Comparator': 'DoesNotExist',
    }
]

### (c) Create and launch the HIT via the API

In [7]:
# Create the HIT
stage1_title = 'Quick 3-question survey about work [<15 seconds], instantly unlock 2nd-stage HIT with higher reward'
stage1_description = 'Quick <15 second survey about workplace conditions, upon submission will receive email with 2nd-stage HIT with higher reward'
stage1_keywords = 'survey,workplace,work'
# Get the reward amount from the .env file
dotenv.load_dotenv(mtglobals.dotenv_fpath, override=True)
stage1_reward = os.getenv("STAGE1_REWARD")

## CAUTION: Only run if you're sure all parameters are set correctly

In [8]:
num_hits = 100

In [9]:
response = client.create_hit(num_hits, stage1_reward, stage1_title, stage1_description,
                             stage1_keywords, stage1_question, stage1_requirements)
# The response includes several fields that will be helpful later
hit_type_id = response['HIT']['HITTypeId']
hit_id = response['HIT']['HITId']
print(f"Created {num_hits} HITs, with HITId: {hit_id}")
print("\nYou can work the HIT here:")
print(client.mturk_environment['preview_url'] + f"?groupId={hit_type_id}")

Created 100 HITs, with HITId: 3TZDZ3Y0JT7UM6OA2VTTFBZ9X0G915

You can work the HIT here:
https://www.mturk.com/mturk/preview?groupId=3IYH2PD9CAK8WJ2510OXIQAXEIJITO


### Now run 01c_MonitorStage1.ipynb

In case of accidental launch, you can use the following code to cancel any not-already-accepted HITs

In [11]:
from datetime import datetime

In [40]:
all_hits = client.boto_client.list_hits()['HITs']

In [41]:
hit_obj = all_hits[0]

In [43]:
hit_id=hit_obj['HITId']

HITId: 3EHIMLB7GKKSFG8JX9I12SH0EU48H4


In [44]:
# Get HIT status
status=client.boto_client.get_hit(HITId=hit_id)['HIT']['HITStatus']
print('HITStatus:', status)

HITStatus: Unassignable


In [45]:
#client.boto_client.get_hit(HITId=hit_id)['HIT']

In [46]:
subs = mtglobals.get_hit_submissions(client.boto_client, hit_id)

In [36]:
from collections import Counter

In [38]:
status_list = []
for cur_sub in subs:
    cur_status = cur_sub['AssignmentStatus']
    status_list.append(cur_status)

In [39]:
status_count = Counter(status_list)
status_count

Counter({'Submitted': 100})

In [47]:
#client.boto_client.list_assignments_for_hit(HITId=hit_id)

In [48]:
# If HIT is active then set it to expire immediately
if status=='Assignable':
    response = client.boto_client.update_expiration_for_hit(
        HITId=hit_id,
        ExpireAt=datetime(2015, 1, 1)
    )

In [49]:
client.boto_client.delete_hit(HITId=hit_id)

RequestError: An error occurred (RequestError) when calling the DeleteHIT operation: This HIT is currently in the state 'Unassignable'.  This operation can be called with a status of: Reviewing, Reviewable (1663803569617)

In [50]:
# Delete the HIT
try:
    client.boto_client.delete_hit(HITId=hit_id)
except:
    print('Not deleted')
else:
    print('Deleted')

Not deleted
